In [1]:
import geopandas as gpd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree


grid_path = r"C:\Users\purav\OneDrive\Desktop\Fi Year Project\Final-Year-Project\Data\Processed\grid_with_data.geojson"
grid = gpd.read_file(grid_path)

c:\Users\purav\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
print(grid.columns)


Index(['FID', 'rainfall_2000', 'rainfall_2001', 'rainfall_2002',
       'rainfall_2003', 'rainfall_2004', 'rainfall_2005', 'rainfall_2006',
       'rainfall_2007', 'rainfall_2008', 'rainfall_2009', 'rainfall_2010',
       'rainfall_2011', 'rainfall_2012', 'rainfall_2013', 'rainfall_2014',
       'rainfall_2015', 'rainfall_2016', 'rainfall_2017', 'rainfall_2018',
       'rainfall_2019', 'rainfall_2020', 'rainfall_2021', 'rainfall_2022',
       'rainfall_2023', 'temperature_2000', 'temperature_2001',
       'temperature_2002', 'temperature_2003', 'temperature_2004',
       'temperature_2005', 'temperature_2006', 'temperature_2007',
       'temperature_2008', 'temperature_2009', 'temperature_2010',
       'temperature_2011', 'temperature_2012', 'temperature_2013',
       'temperature_2014', 'temperature_2015', 'temperature_2016',
       'temperature_2017', 'temperature_2018', 'temperature_2019',
       'temperature_2020', 'temperature_2021', 'temperature_2022',
       'temperature_2023', 

In [5]:
from scipy.spatial import cKDTree
import numpy as np

# Extract centroids as node positions
grid["centroid"] = grid.geometry.centroid
node_positions = np.array([(p.x, p.y) for p in grid["centroid"]])

# Build KDTree for nearest neighbor search
tree = cKDTree(node_positions)
k = 8  # Number of nearest neighbors
distances, indices = tree.query(node_positions, k=k+1)  # +1 to exclude self

C:\Users\purav\AppData\Local\Temp\ipykernel_14624\174489241.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  grid["centroid"] = grid.geometry.centroid


In [7]:
G = nx.Graph()

In [12]:
for year in range(2000, 2024):
    for i in range(len(grid)):
        G.add_node((i, year), 
                   temperature=grid.loc[i, f"temperature_{year}"], 
                   rainfall=grid.loc[i, f"rainfall_{year}"])


In [14]:
for year in range(2000, 2024):
    for i, neighbors in enumerate(indices):
        for j in neighbors[1:]:  # Skip self
            if j >= len(grid):  # Ensure index is valid
                continue

            temp_sim = abs(grid.loc[i, f"temperature_{year}"] - grid.loc[j, f"temperature_{year}"])
            rain_sim = abs(grid.loc[i, f"rainfall_{year}"] - grid.loc[j, f"rainfall_{year}"])
            
            # Spatial weight: temperature has the most impact, then rainfall, then distance
            weight = 1 / (1 + 2 * temp_sim + rain_sim + distances[i][np.where(indices[i] == j)[0][0]])

            # Add edge in the same year
            G.add_edge((i, year), (j, year), weight=weight)


In [15]:
for year in range(2000, 2023):  # From 2000 to 2022 (connecting to next year)
    for i in range(len(grid)):
        temp_diff = abs(grid.loc[i, f"temperature_{year+1}"] - grid.loc[i, f"temperature_{year}"])
        rain_diff = abs(grid.loc[i, f"rainfall_{year+1}"] - grid.loc[i, f"rainfall_{year}"])
        
        # Temporal edges have lower influence than spatial
        weight = 1 / (1 + 0.5 * temp_diff + 0.2 * rain_diff)
        
        # Unidirectional edge (past → future)
        G.add_edge((i, year), (i, year + 1), weight=weight)


In [16]:
print(len(G.nodes), "nodes in the graph")
print(len(G.edges), "edges in the graph")
print(list(G.edges(data=True))[:10])  # Print first 10 edges


120960 nodes in the graph
615576 edges in the graph
[((0, 2000), (126, 2000), {'weight': 0.17767931216190805}), ((0, 2000), (2, 2000), {'weight': 0.40750810292512457}), ((0, 2000), (136, 2000), {'weight': 0.4991863211498181}), ((0, 2000), (124, 2000), {'weight': 0.17264608191373151}), ((0, 2000), (1, 2000), {'weight': 0.41318839614213815}), ((0, 2000), (125, 2000), {'weight': 0.1829518777347174}), ((0, 2000), (4, 2000), {'weight': 0.372857806768115}), ((0, 2000), (134, 2000), {'weight': 0.1552408893377639}), ((0, 2000), (0, 2001), {'weight': 0.6730600862535909}), ((126, 2000), (2, 2000), {'weight': 0.14181570178889755})]
